In [1]:
from pl_test import TextGenerationModel

model = TextGenerationModel.load_from_checkpoint('runs/text_generation/version_1/checkpoints/epoch=44-step=21465.ckpt')

/home/user/workspace/Workspace/seq2seq summary/env/bin/python


In [3]:
model.predict(start_index=11, diversity=0.4, length=400)

' positive sales in Japan, and was praised by both Japanese and western critics. After release, it red t an the the in an a te ingerarithe the f then and con wang in thon a f thing the then therer athed n the te all t he the tin ofont ond the Be Core want thed The toure verin r can ang the me fr an ose thed t se alan he on and te ar thande an the it t to ted ted s wed therere orore ar the ral cond athe bllon arsthe the s ourene men there a as s rtinderin inin tin res s the t ind te we he tereron '